In [3]:
import aurai.data_source as ds
# 根据dsId获取数据源信息
dsId="635241c2adbe1d000a829827"
dsCycle="yyyyMMdd"
ds.describeDS(dsId)

dsId                                         635241c2adbe1d000a829827
name                                                  社会消费品零售总额(杭州数据)
dsType                                                            all
dataType                                                         file
dataSourceName       /data/ai_lab/mlp_web/20221021145239@杭州市数据(1).csv
fileSplitChar                                                       ,
URL                                                              None
datasourceFields    [SJ, SHXFPLSJE, SHJJZB_ZHCK, SHJJZB_KDYWSR, NL...
datasourceCycles                                  [20221021145250907]
dtype: object

In [4]:
import logging
import os
import random
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
#from lstnet_util import GetArguments, LSTNetInit
#from lstnet_datautil import DataUtil
#from lstnet_model import PreSkipTrans, PostSkipTrans, PreARTrans, PostARTrans, LSTNetModel, ModelCompile
#from lstnet_plot import PlotHistory, PlotPrediction
#
import tensorflow as tf
plt.rcParams['font.sans-serif'] = ['SimHei']  # 显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 这两行需要手动设置
logger_name = "lstnet"
log = logging.getLogger(logger_name)
#data = "E:\python\model-lstnet\data"
#output = "E:\python\model-lstnet\output"
data = r"D:\dataset\model-lstnet\data"
output = r"D:\dataset\model-lstnet\output"
#data = r"F:\Coding\Conding\ai\modoul\data"
#output = r"F:\Coding\Conding\ai\modoul\output"

In [5]:
data = ds.load(dsId)
data

,SJ,SHXFPLSJE,SHJJZB_ZHCK,SHJJZB_KDYWSR,NL,XFPLS,JRSC,PPI,ZC,GMJJ
0,时间,社会消费品零售总额,住户存款,快递业务收入,年龄,消费品零售,金融市场,PPI,支出,国民经济
1,2011-03,190.4198,5123.3200,2.4398,2076,1675,2052,4202,1060,1446
2,2011-04,198.7056,5233.5350,2.4959,1991,1661,1993,3993,949,1440
3,2011-05,213.9108,5382.3100,2.4209,2176,1727,2111,3929,991,1397
4,2011-06,206.2171,5454.2816,2.4685,2010,1647,1997,4235,1341,1617
...,...,...,...,...,...,...,...,...,...,...
119,2021-11,681.6318,15557.7544,43.1000,6823,3156,3287,12331,1991,3311
120,2021-12,614.9902,15818.4001,40.8000,6690,3632,3831,10813,2894,3722
121,2022-1+2,1118.5087,31438.3193,52.3000,13900,6216,5538,17657,4610,5632
122,2022-03,507.4263,16899.8900,25.6000,8021,3897,3810,10540,2891,3600


In [6]:
# filename = "D:\dataset\model-lstnet\data\浙江省数据.xlsx" 
# filename="../data/杭州市数据(1).csv"

In [7]:
#data= pd.read_csv(filename, header=0, index_col=0)
head_data=data.head(1) #留下第一行中文行
print(head_data)
data=data.iloc[1:,:]
data

   SJ  SHXFPLSJE SHJJZB_ZHCK SHJJZB_KDYWSR  NL  XFPLS  JRSC  PPI  ZC  GMJJ
0  时间  社会消费品零售总额        住户存款        快递业务收入  年龄  消费品零售  金融市场  PPI  支出  国民经济


,SJ,SHXFPLSJE,SHJJZB_ZHCK,SHJJZB_KDYWSR,NL,XFPLS,JRSC,PPI,ZC,GMJJ
1,2011-03,190.4198,5123.3200,2.4398,2076,1675,2052,4202,1060,1446
2,2011-04,198.7056,5233.5350,2.4959,1991,1661,1993,3993,949,1440
3,2011-05,213.9108,5382.3100,2.4209,2176,1727,2111,3929,991,1397
4,2011-06,206.2171,5454.2816,2.4685,2010,1647,1997,4235,1341,1617
5,2011-07,221.8676,5362.1200,2.4969,2327,1687,1956,4047,936,1445
...,...,...,...,...,...,...,...,...,...,...
119,2021-11,681.6318,15557.7544,43.1000,6823,3156,3287,12331,1991,3311
120,2021-12,614.9902,15818.4001,40.8000,6690,3632,3831,10813,2894,3722
121,2022-1+2,1118.5087,31438.3193,52.3000,13900,6216,5538,17657,4610,5632
122,2022-03,507.4263,16899.8900,25.6000,8021,3897,3810,10540,2891,3600


In [8]:
data.set_index('SJ',inplace=True)
data

,SHXFPLSJE,SHJJZB_ZHCK,SHJJZB_KDYWSR,NL,XFPLS,JRSC,PPI,ZC,GMJJ
SJ,,,,,,,,,
2011-03,190.4198,5123.3200,2.4398,2076,1675,2052,4202,1060,1446
2011-04,198.7056,5233.5350,2.4959,1991,1661,1993,3993,949,1440
2011-05,213.9108,5382.3100,2.4209,2176,1727,2111,3929,991,1397
2011-06,206.2171,5454.2816,2.4685,2010,1647,1997,4235,1341,1617
2011-07,221.8676,5362.1200,2.4969,2327,1687,1956,4047,936,1445
...,...,...,...,...,...,...,...,...,...
2021-11,681.6318,15557.7544,43.1000,6823,3156,3287,12331,1991,3311
2021-12,614.9902,15818.4001,40.8000,6690,3632,3831,10813,2894,3722
2022-1+2,1118.5087,31438.3193,52.3000,13900,6216,5538,17657,4610,5632


In [9]:
import numpy as np
import pandas as pd
# Logging
import logging
logger_name = "lstnet"
log = logging.getLogger(logger_name)

class DataUtil(object):
    #
    # This class contains data specific information.
    # It does the following:
    #  - Read data from file
    #  - Normalise it
    #  - Split it into train, dev (validation) and test
    #  - Create X and Y for each of the 3 sets (train, dev, test) according to the following:
    #    Every sample (x, y) shall be created as follows:
    #     - x --> window number of values
    #     - y --> one value that is at horizon in the future i.e. that is horizon away past the last value of x
    #    This way X and Y will have the following dimensions:
    #     - X [number of samples, window, number of multivariate time series]
    #     - Y [number of samples, number of multivariate time series]
    def __init__(self, filename, train, valid, horizon, window, normalise=2):
        try:
            log.debug("Start reading data")
            self.rawdata =filename
            log.debug("End reading data")
            self.w = window
            self.h = horizon
            self.data = np.zeros(self.rawdata.shape)
            self.n, self.m = self.data.shape
            self.normalise = normalise
            self.scale = np.ones(self.m)
            self.normalise_data(normalise)
            self.split_data(train, valid)
        except IOError as err:
            # In case file is not found, all of the above attributes will not have been created
            # Hence, in order to check if this call was successful, you can call hasattr on this object
            # to check if it has attribute 'data' for example
            log.error("Error opening data file ... %s", err)

    def normalise_data(self, normalise):
        log.debug("Normalise: %d", normalise)
        if normalise == 0:  # do not normalise
            self.data = self.rawdata

        if normalise == 1:  # same normalisation for all timeseries
            self.data = self.rawdata / np.max(self.rawdata)

        if normalise == 2:  # normalise each timeseries alone. This is the default mode
            for i in range(self.m):
                self.scale[i] = np.max(np.abs(self.rawdata[:, i]))
                self.data[:, i] = self.rawdata[:, i] / self.scale[i]

    def split_data(self, train, valid):
        log.info("Splitting data into training set (%.2f), validing set (%.2f) and testing set (%.2f)", train, valid,
                 self.n - (train + valid))

        train_set = range(self.w + self.h - 1, train)  # range(12, 123)
        valid_set = range(train, self.n)  # range(123,135)
        test_set = range(self.n - 3, self.n)

        self.train = self.get_data(train_set)
        self.valid = self.get_data(valid_set)
        self.test = self.get_data(test_set)

    def get_data(self, rng):
        n = len(rng)  # 110、12

        X = np.zeros((n, self.w, self.m))  # X_trian/X_test的shape为（n：样本个数，w：窗口数，m：特征个数）
        Y = np.zeros((n, 1))  # Y_train/Y_test

        for i in range(n):
            end = rng[i] - self.h + 1
            start = end - self.w

            X[i, :, :] = self.data[start:end, :]  # 以w=12为窗口滑动存留数据
            Y[i, 0] = self.data[rng[i], 0]  # 用data的第一列作为标签数据

        return [X, Y]

In [10]:
import os
import numpy as np
import tensorflow as tf

from keras.models import Model, model_from_json
from keras.layers import Input, GRU, Conv2D, Dropout, Flatten, Dense, Reshape, Concatenate, Add
from keras.optimizers import SGD, RMSprop, Adam

from keras import backend as K
# from tensorflow.keras import backend as K
from keras.callbacks import TensorBoard


#######################################################################################################################
#                                      Start Skip RNN specific layers subsclass                                       #
#                                                                                                                     #
# The SkipRNN layer is implemented as follows:                                                                        #
# - Pre Transformation layer that takes a 'window' size of data and apply couple of reshapes and axis transformation  #
#   in order to simulate the skip RNN that is described in the paper                                                  #
# - Apply a normal GRU RNN on the transformed data. This way not the adjacent data points are connected but rather    #
#   data points that are 'skip' away.                                                                                 #
# - Post Transformation layer that brings back dimensions to its original shape as PreTrans has changed all           #
#   dimensions including Batch Size                                                                                   #
#######################################################################################################################

class PreSkipTrans(tf.keras.layers.Layer):
    def __init__(self, pt, skip, **kwargs):
        #
        # pt:   Number of different RNN cells = (window / skip)
        # skip: Number of points to skip
        #
        self.pt = pt
        self.skip = skip
        super(PreSkipTrans, self).__init__(**kwargs)

    def build(self, input_shape):
        super(PreSkipTrans, self).build(input_shape)

    def call(self, inputs):
        # Get input tensors; in this case it's just one tensor
        x = inputs

        # Get the batchsize which is tf.shape(x)[0] since inputs is either X or C which has the same
        # batchsize as the input to the model
        batchsize = tf.shape(x)[0]

        # Get the shape of the input data
        input_shape = K.int_shape(x)

        # Create output data by taking a 'window' size from the end of input (:-self.pt * self.skip)
        output = x[:, -self.pt * self.skip:, :]

        # Reshape the output tensor by:
        # - Changing first dimension (batchsize) from None to the current batchsize
        # - Splitting second dimension into 2 dimensions
        output = tf.reshape(output, [batchsize, self.pt, self.skip, input_shape[2]])

        # Permutate axis 1 and axis 2
        output = tf.transpose(output, perm=[0, 2, 1, 3])

        # Reshape by merging axis 0 and 1 now hence changing the batch size
        # to be equal to current batchsize * skip.
        # This way the time dimension will only contain 'pt' entries which are
        # just values that were originally 'skip' apart from each other => hence skip RNN ready
        output = tf.reshape(output, [batchsize * self.skip, self.pt, input_shape[2]])

        # Adjust the output shape by setting back the batch size dimension to None
        output_shape = tf.TensorShape([None]).concatenate(output.get_shape()[1:])

        return output

    def compute_output_shape(self, input_shape):
        # Since the batch size is None and dimension on axis=2 has not changed,
        # all we need to do is set shape[1] = pt in order to compute the output shape
        shape = tf.TensorShape(input_shape).as_list()
        shape[1] = self.pt

        return tf.TensorShape(shape)

    def get_config(self):
        config = {'pt': self.pt, 'skip': self.skip}
        base_config = super(PreSkipTrans, self).get_config()

        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)


class PostSkipTrans(tf.keras.layers.Layer):
    def __init__(self, skip, **kwargs):
        #
        # skip: Number of points to skip
        #
        self.skip = skip
        super(PostSkipTrans, self).__init__(**kwargs)

    def build(self, input_shape):
        super(PostSkipTrans, self).build(input_shape)

    def call(self, inputs):
        # Get input tensors
        # - First one is the output of the SkipRNN layer which we will operate on
        # - The second is the oiriginal model input tensor which we will use to get
        #   the original batchsize
        x, original_model_input = inputs

        # Get the batchsize which is tf.shape(original_model_input)[0]
        batchsize = tf.shape(original_model_input)[0]

        # Get the shape of the input data
        input_shape = K.int_shape(x)

        # Split the batch size into the original batch size before PreTrans and 'Skip'
        output = tf.reshape(x, [batchsize, self.skip, input_shape[1]])

        # Merge the 'skip' with axis=1
        output = tf.reshape(output, [batchsize, self.skip * input_shape[1]])

        # Adjust the output shape by setting back the batch size dimension to None
        output_shape = tf.TensorShape([None]).concatenate(output.get_shape()[1:])

        return output

    def compute_output_shape(self, input_shape):
        # Adjust shape[1] to be equal to shape[1] * skip in order for the
        # shape to reflect the transformation that was done
        shape = tf.TensorShape(input_shape).as_list()
        shape[1] = self.skip * shape[1]

        return tf.TransformShape(shape)

    def get_config(self):
        config = {'skip': self.skip}
        base_config = super(PostSkipTrans, self).get_config()

        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)


#######################################################################################################################
#                                      End Skip RNN specific layers subsclass                                         #
#######################################################################################################################


#######################################################################################################################
#                                      Start AR specific layers subsclass                                             #
#                                                                                                                     #
# -                                                                          #
# - Pre Transformation layer that takes a 'highway' size of data and apply a reshape and axis transformation          #
# - Flatten the output and pass it through a Dense layer with one output                                              #
# - Post Transformation layer that bring back dimensions to its original shape                                        #                                                     #
#######################################################################################################################

class PreARTrans(tf.keras.layers.Layer):
    def __init__(self, hw, **kwargs):
        #
        # hw: Highway = Number of timeseries values to consider for the linear layer (AR layer)
        #
        self.hw = hw
        super(PreARTrans, self).__init__(**kwargs)

    def build(self, input_shape):
        super(PreARTrans, self).build(input_shape)

    def call(self, inputs):
        # Get input tensors; in this case it's just one tensor: X = the input to the model
        x = inputs

        # Get the batchsize which is tf.shape(x)[0]
        batchsize = tf.shape(x)[0]

        # Get the shape of the input data
        input_shape = K.int_shape(x)

        # Select only 'highway' length of input to create output
        output = x[:, -self.hw:, :]

        # Permute axis 1 and 2. axis=2 is the the dimension having different time-series
        # This dimension should be equal to 'm' which is the number of time-series.
        output = tf.transpose(output, perm=[0, 2, 1])

        # Merge axis 0 and 1 in order to change the batch size
        output = tf.reshape(output, [batchsize * input_shape[2], self.hw])

        # Adjust the output shape by setting back the batch size dimension to None
        output_shape = tf.TensorShape([None]).concatenate(output.get_shape()[1:])

        return output

    def compute_output_shape(self, input_shape):
        # Set the shape of axis=1 to be hw since the batchsize is NULL
        shape = tf.TensorShape(input_shape).as_list()
        shape[1] = self.hw

        return tf.TensorShape(shape)

    def get_config(self):
        config = {'hw': self.hw}
        base_config = super(PreARTrans, self).get_config()

        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)


class PostARTrans(tf.keras.layers.Layer):
    def __init__(self, m, **kwargs):
        #
        # m: Number of timeseries
        #
        self.m = m
        super(PostARTrans, self).__init__(**kwargs)

    def build(self, input_shape):
        super(PostARTrans, self).build(input_shape)

    def call(self, inputs):
        # Get input tensors
        # - First one is the output of the Dense(1) layer which we will operate on
        # - The second is the oiriginal model input tensor which we will use to get
        #   the original batchsize
        x, original_model_input = inputs

        # Get the batchsize which is tf.shape(original_model_input)[0]
        batchsize = tf.shape(original_model_input)[0]

        # Get the shape of the input data
        input_shape = K.int_shape(x)

        # Reshape the output to have the batch size equal to the original batchsize before PreARTrans
        # and the second dimension as the number of timeseries
        output = tf.reshape(x, [batchsize, self.m])

        # Adjust the output shape by setting back the batch size dimension to None
        output_shape = tf.TensorShape([None]).concatenate(output.get_shape()[1:])

        return output

    def compute_output_shape(self, input_shape):
        # Adjust shape[1] to be equal 'm'
        shape = tf.TensorShape(input_shape).as_list()
        shape[1] = self.m

        return tf.TensorShape(shape)

    def get_config(self):
        config = {'m': self.m}
        base_config = super(PostARTrans, self).get_config()

        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        return cls(**config)


#######################################################################################################################
#                                      End AR specific layers subsclass                                               #
#######################################################################################################################

#######################################################################################################################
#                                                 Model Start                                                         #
#                                                                                                                     #
# The model, as per the paper has the following layers:                                                               #
# - CNN                                                                                                               #
# - GRU                                                                                                               #
# - SkipGRU                                                                                                           #
# - AR                                                                                                                #
#######################################################################################################################

def LSTNetModel(init, input_shape):
    # m is the number of time-series
    m = input_shape[2]

    # Get tensor shape except batchsize
    tensor_shape = input_shape[1:]

    if K.image_data_format() == 'channels_last':
        ch_axis = 3
    else:
        ch_axis = 1

    X = Input(shape=tensor_shape)

    # CNN
    if init.CNNFilters > 0 and init.CNNKernel > 0:
        # Add an extra dimension of size 1 which is the channel dimension in Conv2D
        C = Reshape((input_shape[1], input_shape[2], 1))(X)

        # Apply a Conv2D that will transform it into data of dimensions (batchsize, time, 1, NumofFilters)
        C = Conv2D(filters=init.CNNFilters, kernel_size=(init.CNNKernel, m), kernel_initializer=init.initialiser)(C)
        C = Dropout(init.dropout)(C)

        # Adjust data dimensions by removing axis=2 which is always equal to 1
        c_shape = K.int_shape(C)
        C = Reshape((c_shape[1], c_shape[3]))(C)
    else:
        # If configured not to apply CNN, copy the input
        C = X

    # GRU
    # Apply a GRU layer (with activation set to 'relu' as per the paper) and take the returned states as result
    _, R = GRU(init.GRUUnits, activation="relu", return_sequences=False, return_state=True)(C)
    R = Dropout(init.dropout)(R)

    # SkipGRU
    if init.skip > 0:
        # Calculate the number of values to use which is equal to the window divided by how many time values to skip
        pt = int(init.window / init.skip)

        S = PreSkipTrans(pt, int((init.window - init.CNNKernel + 1) / pt))(C)
        _, S = GRU(init.SkipGRUUnits, activation="relu", return_sequences=False, return_state=True)(S)
        S = PostSkipTrans(int((init.window - init.CNNKernel + 1) / pt))([S, X])

        # Concatenate the outputs of GRU and SkipGRU
        R = Concatenate(axis=1)([R, S])

    # Dense layer
    Y = Flatten()(R)
    Y = Dense(m)(Y)

    # AR
    if init.highway > 0:
        Z = PreARTrans(init.highway)(X)
        Z = Flatten()(Z)
        Z = Dense(1)(Z)
        Z = PostARTrans(m)([Z, X])

        # Generate output as the summation of the Dense layer output and the AR one
        Y = Add()([Y, Z])

    # Generate Model
    model = Model(inputs=X, outputs=Y)

    return model


#######################################################################################################################
#                                                 Model End                                                           #
#######################################################################################################################

#######################################################################################################################
#                                                 Model Utilities                                                     #
#                                                                                                                     #
# Below is a collection of functions:                                                                                 #
# - rse:         A metrics function that calculates the root square error                                             #
# - corr:        A metrics function that calculates the correlation                                                   #
#######################################################################################################################
def rse(y_true, y_pred):
    #
    # The formula is:
    #           K.sqrt(K.sum(K.square(y_true - y_pred)))
    #    RSE = -----------------------------------------------
    #           K.sqrt(K.sum(K.square(y_true_mean - y_true)))
    #
    #           K.sqrt(K.sum(K.square(y_true - y_pred))/(N-1))
    #        = ----------------------------------------------------
    #           K.sqrt(K.sum(K.square(y_true_mean - y_true)/(N-1)))
    #
    #
    #           K.sqrt(K.mean(K.square(y_true - y_pred)))
    #        = ------------------------------------------
    #           K.std(y_true)
    #
    num = K.sqrt(K.mean(K.square(y_true - y_pred), axis=None))
    den = K.std(y_true, axis=None)

    return num / den


def corr(y_true, y_pred):
    #
    # This function calculates the correlation between the true and the predicted outputs
    #
    num1 = y_true - K.mean(y_true, axis=0)
    num2 = y_pred - K.mean(y_pred, axis=0)

    num = K.mean(num1 * num2, axis=0)
    den = K.std(y_true, axis=0) * K.std(y_pred, axis=0)

    return K.mean(num / den)


def MAE(y_true, y_pred):
    return K.mean(K.abs(y_true - y_pred))


def MSE(y_true, y_pred):
    return K.mean((y_true - y_pred) ** 2)


def RMSE(y_true, y_pred):
    return K.sqrt(MSE(y_true, y_pred))


def MAPE(y_true, y_pred):
    return K.mean(K.abs((y_true - y_pred) / y_pred))


def MSPE(y_true, y_pred):
    return K.mean(K.square((y_true - y_pred) / y_pred))
#
# A function that compiles 'model' after setting the appropriate:
# - optimiser function passed via init
# - learning rate passed via init
# - loss function also set in init
# - metrics
#
def ModelCompile(model, init):
    # Select the appropriate optimiser and set the learning rate from input values (arguments)
    if init.optimiser == "SGD":
        opt = SGD(lr=init.lr, momentum=0.0, decay=0.0, nesterov=False)
    elif init.optimiser == "RMSprop":
        opt = RMSprop(lr=init.lr, rho=0.9, epsilon=None, decay=0.0)
    else:  # Adam
        opt = Adam(lr=init.lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

    # Compile using the previously defined metrics
    model.compile(optimizer=opt, loss=init.loss, metrics=[rse, corr, MAPE, RMSE])

    # Launch Tensorboard if selected in arguments
    if init.tensorboard != None:
        tensorboard = TensorBoard(log_dir=init.tensorboard)
    else:
        tensorboard = None

    return tensorboard

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import logging
plt.rcParams['font.sans-serif'] = ['SimHei']  # 显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 这两行需要手动设置
logger_name = "lstnet"
log = logging.getLogger(logger_name)


def PlotHistory(history, metrics, init):
    if history is not None and metrics is not None and init is not None:
        log.info("Plotting history ...")
        # Number of keys present in the history dictionary
        i = 1
        #
        # Number of metrics that were trained and are available in history
        # This will help us determine the width of the canvas as well as correctly set
        # the parameters to subplot
        #
        n = len(history)
        #
        # The number of rows is set so that the training results are on one line and the
        # validation ones are on the second. Therefore:
        #         number of available metrics in history
        # rows = --------------------------------------- = 2 in case of validate=True. Otherwise 1
        #         number of metrics
        #
        rows = int(n / len(metrics))

        #
        # Number of columns i.e. number of different metrics plotted for each of the training and validation
        #
        cols = int(n / rows)

        #
        # Set the plotting image size
        # If the number of columns is greater than 2, choose 16, otherwise 12
        # If the number of rows is greater than 1, choose 10, otherwise 5
        #
        fig = plt.figure(figsize=(16 if cols > 2 else 12, 10 if rows > 1 else 5))

        # Training data history plot
        for m in metrics:
            key = m
            log.debug("Plotting metrics %s", key)
            plt.subplot(rows, cols, i)
            plt.plot(history[key])
            plt.ylabel(m.title())
            plt.xlabel("Epochs")
            plt.title("Training " + m.title())
            i += 1

        # Validation data history plot (if available)
        for m in metrics:
            key = "val_" + m
            log.debug("Plotting metrics %s", key)
            # if key is not in history.keys() => --validate was set to False and therefore history for validation is
            # not available
            if key in history.keys():
                plt.subplot(rows, cols, i)
                plt.plot(history[key])
                plt.ylabel(m.title())
                plt.xlabel("Epochs")
                plt.title("Validation " + m.title())
                i += 1

        fig.canvas.set_window_title('Training History')
        plt.show()


def PlotPrediction(Data, init, trainPredict, validPredict):
    if Data is not None and init is not None:
        log.info("Plotting Prediction ...")
        series = 0
        log.debug("Initialising trainPredictPlot, ValidPredictPlot, testPredictPlot")
        trainPredictPlot = np.empty((Data.n, Data.m))
        trainPredictPlot[:, :] = np.nan
        validPredictPlot = np.empty((Data.n, Data.m))
        validPredictPlot[:, :] = np.nan

        start = init.window + init.horizon - 1
        end = start + len(Data.train[0])  # Same length as trainPredict however we might not have trainPredict
        if trainPredict is not None:
            log.debug("Filling trainPredictPlot from %d to %d", start, end)
            trainPredictPlot[start:end, :] = trainPredict

        start = end
        end = start + len(Data.valid[0])  # Same length as validPredict however we might not have validPredict
        if validPredict is not None:
            log.debug("Filling validPredictPlot from %d to %d", start, end)
            validPredictPlot[start:end, :] = validPredict

        fig = plt.figure()

        plt.plot(Data.data[:, series], c='b', label="Train set (actual)")
        plt.plot(trainPredictPlot[:, series], c='g', label="Train set (predict)")
        plt.plot(validPredictPlot[:, series], c='y', label="Test set (predict)")

        plt.ylabel("Timeseries")
        plt.xlabel("Time")
        plt.title("Prediction Plotting for timeseries # %s" % "社会消费品零售总额(亿元)")
        plt.legend()
        fig.canvas.set_window_title('Prediction')
        plt.show()

In [12]:
import logging
import os
import random
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei']  # 显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 这两行需要手动设置

In [13]:
import argparse


class LSTNetInit(object):
    def __init__(self, args, args_is_dictionary=False):
        if args_is_dictionary is True:
            self.window = args["window"]
            self.horizon = args["horizon"]
            self.CNNFilters = args["CNNFilters"]
            self.CNNKernel = args["CNNKernel"]
            self.GRUUnits = args["GRUUnits"]
            self.SkipGRUUnits = args["SkipGRUUnits"]
            self.skip = args["skip"]
            self.dropout = args["dropout"]
            self.normalise = args["normalize"]
            self.highway = args["highway"]
            self.batchsize = args["batchsize"]
            self.epochs = args["epochs"]
            self.initialiser = args["initializer"]
            self.trainpercent = args["trainpercent"]
            self.validpercent = args["validpercent"]
            self.loss = args["loss"]
            self.lr = args["lr"]
            self.optimiser = args["optimizer"]
            self.tensorboard = args["tensorboard"]
            # self.predict = args["predict"]

        else:
            self.window = args.window
            self.horizon = args.horizon
            self.CNNFilters = args.CNNFilters
            self.CNNKernel = args.CNNKernel
            self.GRUUnits = args.GRUUnits
            self.SkipGRUUnits = args.SkipGRUUnits
            self.skip = args.skip
            self.dropout = args.dropout
            self.normalise = args.normalize
            self.highway = args.highway
            self.batchsize = args.batchsize
            self.epochs = args.epochs
            self.initialiser = args.initializer
            self.trainpercent = args.trainpercent
            self.validpercent = args.validpercent
            self.loss = args.loss
            self.lr = args.lr
            self.optimiser = args.optimizer
            self.tensorboard = args.tensorboard
            # self.predict = args.predict

    def dump(self):
        import logging
        logger_name = "lstnet"
        log = logging.getLogger(logger_name)
        log.debug("Window: %d", self.window)
        log.debug("Horizon: %d", self.horizon)
        log.debug("CNN Filters: %d", self.CNNFilters)
        log.debug("CNN Kernel: %d", self.CNNKernel)
        log.debug("GRU Units: %d", self.GRUUnits)
        log.debug("Skip GRU Units: %d", self.SkipGRUUnits)
        log.debug("Skip: %d", self.skip)
        log.debug("Dropout: %f", self.dropout)
        log.debug("Normalise: %d", self.normalise)
        log.debug("Highway: %d", self.highway)
        log.debug("Batch size: %d", self.batchsize)
        log.debug("Epochs: %d", self.epochs)


def GetArguments():
    # Creating the argument parser
    parser = argparse.ArgumentParser(description='LSTNet Model')
    parser.add_argument('--window', type=int, default=11 * 1, help='Window size. Default=24*7')
    parser.add_argument('--horizon', type=int, default=1, help='Horizon width. Default=12')
    parser.add_argument('--CNNFilters', type=int, default=100,
                        help='Number of CNN layer filters. Default=100. If set to 0, the CNN layer will be omitted')
    parser.add_argument('--CNNKernel', type=int, default=3,
                        help='Size of the CNN filters. Default=6. If set to 0, the CNN layer will be omitted')
    parser.add_argument('--GRUUnits', type=int, default=100, help='Number of GRU hidden units. Default=100')
    parser.add_argument('--SkipGRUUnits', type=int, default=3,
                        help='Number of hidden units in the Skip GRU layer. Default=5')
    parser.add_argument('--skip', type=int, default=6,
                        help='Size of the window to skip in the Skip GRU layer. Default=24. If set to 0, the SkipGRU layer will be omitted')
    parser.add_argument('--dropout', type=float, default=0.1,
                        help='Dropout to be applied to layers. 0 means no dropout. Default=0.2')
    parser.add_argument('--normalize', type=int, default=2,
                        help='0 = do not normalise, 1 = use same normalisation for all timeseries, 2 = normalise each timeseries independently. Default=2')
    parser.add_argument('--highway', type=int, default=1,
                        help='The window size of the highway component. Default=24. If set to 0, the AR layer will be omitted')
    parser.add_argument('--lr', type=float, default=0.0003, help='Learning rate. Default=0.001')
    parser.add_argument('--batchsize', type=int, default=24, help='Training batchsize. Default=128')
    parser.add_argument('--epochs', type=int, default=1000, help='Number of epochs to run for training. Default=100')
    parser.add_argument('--initializer', type=str, default="glorot_uniform",
                        help='Weights initialiser to use. Default=glorot_uniform')
    parser.add_argument('--loss', type=str, default="mean_absolute_error",
                        help='Loss function to use. Default=mean_absolute_error')
    parser.add_argument('--optimizer', type=str, default="Adam", help='Optimisation function to use. Default=Adam')
    parser.add_argument('--trainpercent', type=int, default=117,
                        help='111 / 117 ')
    parser.add_argument('--validpercent', type=int, default=10,
                        help='Percentage of data to be used for validation. Default=0.2')
    parser.add_argument('--tensorboard', type=str, default=None,
                        help='Location of the tensorboard folder. If not set, tensorboard will not be launched. Default=None i.e. no tensorboard callback')

    args = parser.parse_args(args=[])
    return args


In [14]:
logger_name = "lstnet"
log = logging.getLogger(logger_name)

In [15]:
SEED = 2022
def set_seeds(seed=SEED):
        os.environ['PYTHONHASHSEED'] = str(seed)
        random.seed(seed)
        tf.random.set_seed(seed)
        #tf.set_random_seed(seed)
        np.random.seed(seed)


def set_global_determinism(seed=SEED):
        set_seeds(seed=seed)

        os.environ['TF_DETERMINISTIC_OPS'] = '1'
        os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

        #tf.config.threading.set_inter_op_parallelism_threads(1)
        #tf.config.threading.set_intra_op_parallelism_threads(1)

set_global_determinism(seed=SEED)

In [16]:
def train(model, data, init):
       val_data = (data.valid[0], data.valid[1])
       callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)
       start_time = datetime.now()
       history = model.fit(
           x=data.train[0],
           y=data.train[1],
           epochs=init.epochs,
           batch_size=init.batchsize,
           validation_data=val_data,
           # callbacks=[callback]
       )
       end_time = datetime.now()
       log.info("Training time took: %s", str(end_time - start_time))
       model.summary()
       return history

In [17]:
datavalues=data.values.astype("float")
print(type(datavalues))
datavalues

<class 'numpy.ndarray'>


array([[1.90419800e+02, 5.12332000e+03, 2.43980000e+00, ...,
        4.20200000e+03, 1.06000000e+03, 1.44600000e+03],
       [1.98705600e+02, 5.23353500e+03, 2.49590000e+00, ...,
        3.99300000e+03, 9.49000000e+02, 1.44000000e+03],
       [2.13910800e+02, 5.38231000e+03, 2.42090000e+00, ...,
        3.92900000e+03, 9.91000000e+02, 1.39700000e+03],
       ...,
       [1.11850870e+03, 3.14383193e+04, 5.23000000e+01, ...,
        1.76570000e+04, 4.61000000e+03, 5.63200000e+03],
       [5.07426300e+02, 1.68998900e+04, 2.56000000e+01, ...,
        1.05400000e+04, 2.89100000e+03, 3.60000000e+03],
       [5.04048200e+02, 1.68856659e+04, 2.37000000e+01, ...,
        1.02030000e+04, 2.59100000e+03, 3.46500000e+03]])

In [18]:

args = GetArguments()
lstnet_init = LSTNetInit(args)
Data = DataUtil(datavalues,
                    lstnet_init.trainpercent,
                    lstnet_init.validpercent,
                    lstnet_init.horizon,
                    lstnet_init.window,
                    lstnet_init.normalise)

In [19]:
print(Data.train[0])

[[[0.17024436 0.16296418 0.0466501  ... 0.23797927 0.18714689 0.25297411]
  [0.17765226 0.16646994 0.04772275 ... 0.22614261 0.16754944 0.25192442]
  [0.19124643 0.17120222 0.04628872 ... 0.22251798 0.17496469 0.24440168]
  ...
  [0.20908537 0.17039473 0.06193881 ... 0.23746956 0.15307203 0.25822253]
  [0.23056575 0.18043176 0.0666501  ... 0.2471541  0.17461158 0.28936319]
  [0.41425766 0.37065849 0.10390249 ... 0.42951804 0.35169492 0.46606018]]

 [[0.17765226 0.16646994 0.04772275 ... 0.22614261 0.16754944 0.25192442]
  [0.19124643 0.17120222 0.04628872 ... 0.22251798 0.17496469 0.24440168]
  [0.1843679  0.17349151 0.04719885 ... 0.23984822 0.23675847 0.28289013]
  ...
  [0.23056575 0.18043176 0.0666501  ... 0.2471541  0.17461158 0.28936319]
  [0.41425766 0.37065849 0.10390249 ... 0.42951804 0.35169492 0.46606018]
  [0.19333904 0.18131472 0.06672275 ... 0.23220253 0.25547316 0.26329601]]

 [[0.19124643 0.17120222 0.04628872 ... 0.22251798 0.17496469 0.24440168]
  [0.1843679  0.173491

In [20]:


# Compile model
lstnet = LSTNetModel(lstnet_init, Data.train[0].shape)
lstnet_tensorboard = ModelCompile(lstnet, lstnet_init)
# Model Training
h = train(lstnet, Data, lstnet_init)
# Testing evaluation
print(
        '| loss{:5.4f} | rse{:5.4f} | corr{:5.4f} | MAPE{:5.4f} | RMSE{:5.4f} \n| test_loss{:5.4f} | test_rse{:5.4f} | '
        'test_corr{:5.4f} | test_MAPE{:5.4f} | test_RMSE{:5.4f}'.format(
            (h.history['loss'][-1]), (h.history['rse'][-1]), (h.history['corr'][-1]),
            (h.history['MAPE'][-1]), (h.history['RMSE'][-1]), (h.history['val_loss'])[-1],
            (h.history['val_rse'][-1]), (h.history['val_corr'][-1]), (h.history['val_MAPE'][-1]),
            (h.history['val_RMSE'][-1])))

Epoch 1/1000


/usr/local/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1/5 [=====>........................] - ETA: 19s - loss: 0.4003 - rse: 3.1970 - corr: 0.1046 - MAPE: 22.2631 - RMSE: 0.4249

5/5 [==============================] - ETA: 0s - loss: 0.3615 - rse: 3.4740 - corr: 0.1932 - MAPE: 21.2115 - RMSE: 0.3830 

5/5 [==============================] - 6s 242ms/step - loss: 0.3615 - rse: 3.4740 - corr: 0.1932 - MAPE: 21.2115 - RMSE: 0.3830 - val_loss: 0.4952 - val_rse: 2.8809 - val_corr: -0.3557 - val_MAPE: 15.2260 - val_RMSE: 0.5353


...

Total params: 65,499


Trainable params: 65,499


Non-trainable params: 0


__________________________________________________________________________________________________


| loss0.0109 | rse0.1193 | corr0.9940 | MAPE0.0264 | RMSE0.0152 
| test_loss0.0485 | test_rse0.3220 | test_corr0.9637 | test_MAPE0.0789 | test_RMSE0.0598


In [21]:
train_predict = lstnet.predict(Data.train[0]) * Data.scale
valid_predict = lstnet.predict(Data.valid[0]) * Data.scale

1/4 [======>.......................] - ETA: 1s

4/4 [==============================] - 1s 6ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 40ms/step


In [22]:
# Prediction--saved
raw_data = list(Data.rawdata[:, 0])
pret_data = [None for _ in range(11)] + [x for x in train_predict[:, 0]] + [x for x in valid_predict[:, 0]]
#data_name = {"原始数据": raw_data,"预测值": pret_data}
#save_data = pd.DataFrame(data_name)
#save_data.to_csv(output + '预测--XX.csv', encoding='utf-8')

In [23]:
# Prediction-future
def get_data(new_data, rng):
    n = len(rng)  # 110、12
    X = np.zeros((n, lstnet_init.window, Data.m))  # X_trian/X_test的shape为（n：样本个数，w：窗口数，m：特征个数）
    for i in range(n):
        end = rng[i] + 1
        start = end - lstnet_init.window
        X[i, :, :] = new_data[start:end, :]  # 以w=11为窗口滑动存留数据
    return X
new_data = Data.data

In [24]:
# 起始索引
start_indxe = len(new_data) -1 
# 预测周期
predict_windows = 12 
print(start_indxe)

122


In [25]:
for step in range(start_indxe, start_indxe+predict_windows):
    rng = range(step, step + 1)
    test_X = get_data(new_data, rng=rng)
    testPredict = lstnet.predict(test_X)  # 向后的第一期预测
    test_predict = lstnet.predict(test_X) * Data.scale  #
    print(test_predict[:, 0],new_data.shape,testPredict.shape)
    new_data = np.vstack((new_data, testPredict))

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 37ms/step


[639.42246393] (123, 9) (1, 9)


...

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 39ms/step


[555.89233361] (134, 9) (1, 9)


In [26]:
result_df = pd.DataFrame(new_data* Data.scale,columns=data.columns)
result_df
save_data = pd.DataFrame(result_df)
save_data.to_csv(output + '预测--XX.csv', encoding='utf-8')
#Index(['社会消费品零售总额', '住户存款', '住户短期贷款', '快递业务收入', '年龄', '消费品零售', '金融市场', 'PPI','支出', '国民经济'],dtype='object')
#data.columns
#new_data

In [27]:
data.index.values

array(['2011-03', '2011-04', '2011-05', '2011-06', '2011-07', '2011-08',
       '2011-09', '2011-10', '2011-11', '2011-12', '2012-1+2', '2012-03',
       '2012-04', '2012-05', '2012-06', '2012-07', '2012-08', '2012-09',
       '2012-10', '2012-11', '2012-12', '2013-1+2', '2013-03', '2013-04',
       '2013-05', '2013-06', '2013-07', '2013-08', '2013-09', '2013-10',
       '2013-11', '2013-12', '2014-1+2', '2014-03', '2014-04', '2014-05',
       '2014-06', '2014-07', '2014-08', '2014-09', '2014-10', '2014-11',
       '2014-12', '2015-1+2', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-1+2', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07',
       '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-1+2',
       '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08',
       '2017-09', '2017-10', '2017-11', '2017-12', '2018-1+2', '2018-03',
       '2018-04', '2018-05', '2018-06', '201

In [28]:
data.index.values[-predict_windows:]

array(['2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09',
       '2021-10', '2021-11', '2021-12', '2022-1+2', '2022-03', '2022-04'],
      dtype=object)

In [29]:
next_period_index = pd.DataFrame(data.index.values[-predict_windows:],columns=['p_mon'])

In [30]:
next_period_index['p_year'] = next_period_index['p_mon'].apply(lambda x:x.split('-')[0])
next_period_index['next_2_year'] = next_period_index['p_mon'].apply(lambda x:str(int(x.split('-')[0])+2))
next_period_index['next_year'] = next_period_index['p_mon'].apply(lambda x:str(int(x.split('-')[0])+1))

next_period_index['mon_id'] = next_period_index['p_mon'].apply(lambda x:x.split('-')[1])
next_period_index['next_p_mon'] = next_period_index['next_year']+'-'+next_period_index['mon_id']
next_period_index['next_2_p_mon'] = next_period_index['next_2_year']+'-'+next_period_index['mon_id']

#next_period_index_array = next_period_index_array.append(next_period_index['next_2_p_mon'].values)
next_period_index

,p_mon,p_year,next_2_year,next_year,mon_id,next_p_mon,next_2_p_mon
0,2021-04,2021,2023,2022,04,2022-04,2023-04
1,2021-05,2021,2023,2022,05,2022-05,2023-05
2,2021-06,2021,2023,2022,06,2022-06,2023-06
3,2021-07,2021,2023,2022,07,2022-07,2023-07
4,2021-08,2021,2023,2022,08,2022-08,2023-08
5,2021-09,2021,2023,2022,09,2022-09,2023-09
6,2021-10,2021,2023,2022,10,2022-10,2023-10
7,2021-11,2021,2023,2022,11,2022-11,2023-11
8,2021-12,2021,2023,2022,12,2022-12,2023-12
9,2022-1+2,2022,2024,2023,1+2,2023-1+2,2024-1+2


In [31]:
#next_period_index_array

In [32]:
new_period=list(data.index.tolist()+next_period_index['next_p_mon'].tolist()+next_period_index['next_2_p_mon'].tolist())
new_period

['2011-03',
 '2011-04',
 '2011-05',
 '2011-06',
 '2011-07',
 '2011-08',
 '2011-09',
 '2011-10',
 '2011-11',
 '2011-12',
 '2012-1+2',
 '2012-03',
 '2012-04',
 '2012-05',
 '2012-06',
 '2012-07',
 '2012-08',
 '2012-09',
 '2012-10',
 '2012-11',
 '2012-12',
 '2013-1+2',
 '2013-03',
 '2013-04',
 '2013-05',
 '2013-06',
 '2013-07',
 '2013-08',
 '2013-09',
 '2013-10',
 '2013-11',
 '2013-12',
 '2014-1+2',
 '2014-03',
 '2014-04',
 '2014-05',
 '2014-06',
 '2014-07',
 '2014-08',
 '2014-09',
 '2014-10',
 '2014-11',
 '2014-12',
 '2015-1+2',
 '2015-03',
 '2015-04',
 '2015-05',
 '2015-06',
 '2015-07',
 '2015-08',
 '2015-09',
 '2015-10',
 '2015-11',
 '2015-12',
 '2016-1+2',
 '2016-03',
 '2016-04',
 '2016-05',
 '2016-06',
 '2016-07',
 '2016-08',
 '2016-09',
 '2016-10',
 '2016-11',
 '2016-12',
 '2017-1+2',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-1+2',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08

In [33]:
len(new_period)

147

In [34]:
result_df['p_year'] = new_period[:len(result_df)]

In [35]:
result_df

,SHXFPLSJE,SHJJZB_ZHCK,SHJJZB_KDYWSR,NL,XFPLS,JRSC,PPI,ZC,GMJJ,p_year
0,190.419800,5123.320000,2.439800,2076.000000,1675.000000,2052.000000,4202.000000,1060.000000,1446.000000,2011-03
1,198.705600,5233.535000,2.495900,1991.000000,1661.000000,1993.000000,3993.000000,949.000000,1440.000000,2011-04
2,213.910800,5382.310000,2.420900,2176.000000,1727.000000,2111.000000,3929.000000,991.000000,1397.000000,2011-05
3,206.217100,5454.281600,2.468500,2010.000000,1647.000000,1997.000000,4235.000000,1341.000000,1617.000000,2011-06
4,221.867600,5362.120000,2.496900,2327.000000,1687.000000,1956.000000,4047.000000,936.000000,1445.000000,2011-07
...,...,...,...,...,...,...,...,...,...,...
130,576.456642,16155.107976,27.016787,7127.802879,3309.255110,4036.018261,9176.072260,2927.802641,2919.855134,2022-11
131,908.927922,25600.736867,41.794282,11339.802426,5134.427220,6271.701646,14399.035495,4653.144482,4629.043530,2022-12
132,493.256319,14147.715525,23.329577,6226.058513,2858.343096,3412.713977,7881.453102,2499.674303,2528.203618,2023-1+2
133,541.539512,15069.279098,24.944151,6640.681136,3100.892048,3698.994235,8524.230352,2743.246124,2748.920248,2023-03


In [36]:
result_df.insert(0,'p_year',result_df.pop('p_year'))
result_df

,p_year,SHXFPLSJE,SHJJZB_ZHCK,SHJJZB_KDYWSR,NL,XFPLS,JRSC,PPI,ZC,GMJJ
0,2011-03,190.419800,5123.320000,2.439800,2076.000000,1675.000000,2052.000000,4202.000000,1060.000000,1446.000000
1,2011-04,198.705600,5233.535000,2.495900,1991.000000,1661.000000,1993.000000,3993.000000,949.000000,1440.000000
2,2011-05,213.910800,5382.310000,2.420900,2176.000000,1727.000000,2111.000000,3929.000000,991.000000,1397.000000
3,2011-06,206.217100,5454.281600,2.468500,2010.000000,1647.000000,1997.000000,4235.000000,1341.000000,1617.000000
4,2011-07,221.867600,5362.120000,2.496900,2327.000000,1687.000000,1956.000000,4047.000000,936.000000,1445.000000
...,...,...,...,...,...,...,...,...,...,...
130,2022-11,576.456642,16155.107976,27.016787,7127.802879,3309.255110,4036.018261,9176.072260,2927.802641,2919.855134
131,2022-12,908.927922,25600.736867,41.794282,11339.802426,5134.427220,6271.701646,14399.035495,4653.144482,4629.043530
132,2023-1+2,493.256319,14147.715525,23.329577,6226.058513,2858.343096,3412.713977,7881.453102,2499.674303,2528.203618
133,2023-03,541.539512,15069.279098,24.944151,6640.681136,3100.892048,3698.994235,8524.230352,2743.246124,2748.920248


In [37]:
result_data=result_df[['SHXFPLSJE']]
result_data.insert(0, 'datatime-x', result_df['p_year'].values)
result_data.insert(2, 'YCZ', result_df['SHXFPLSJE'].values)
result_data

,datatime-x,SHXFPLSJE,YCZ
0,2011-03,190.419800,190.419800
1,2011-04,198.705600,198.705600
2,2011-05,213.910800,213.910800
3,2011-06,206.217100,206.217100
4,2011-07,221.867600,221.867600
...,...,...,...
130,2022-11,576.456642,576.456642
131,2022-12,908.927922,908.927922
132,2023-1+2,493.256319,493.256319
133,2023-03,541.539512,541.539512


In [38]:
for i in range(len(result_data)-1,len(result_data)-predict_windows-1,-1):
    result_data.loc[i,'SHXFPLSJE']=None
result_data
# print(len(result_data))

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,datatime-x,SHXFPLSJE,YCZ
0,2011-03,190.4198,190.419800
1,2011-04,198.7056,198.705600
2,2011-05,213.9108,213.910800
3,2011-06,206.2171,206.217100
4,2011-07,221.8676,221.867600
...,...,...,...
130,2022-11,NaN,576.456642
131,2022-12,NaN,908.927922
132,2023-1+2,NaN,493.256319
133,2023-03,NaN,541.539512


In [39]:
#新建一个df
df2 = pd.DataFrame(['时间','社会消费品零售总额','预测值']).T
# 修改df2的column和df的一致
df2.columns = result_data.columns
df2

,datatime-x,SHXFPLSJE,YCZ
0,时间,社会消费品零售总额,预测值


In [40]:
df = pd.concat([df2,result_data], axis=0 ,ignore_index=True)
df

,datatime-x,SHXFPLSJE,YCZ
0,时间,社会消费品零售总额,预测值
1,2011-03,190.42,190.42
2,2011-04,198.706,198.706
3,2011-05,213.911,213.911
4,2011-06,206.217,206.217
...,...,...,...
131,2022-11,NaN,576.457
132,2022-12,NaN,908.928
133,2023-1+2,NaN,493.256
134,2023-03,NaN,541.54


In [41]:
df.to_csv(output + '预测3--XX.csv', encoding='utf_8_sig')